In [5]:
from utils import ConvertParserOutput

In [4]:
store_path = Path('./database/decoded_text_across_parsers.csv')
fill_na_with_empty_str:bool = False

assert store_path.parent.is_dir(), "Directory does exist"

# def creat_text_database (rows: pdf files, column parser)
all_dict = {}

# list all parsers (html = groundtruth)
parser_name_list = ['html', 'marker', 'nougat', 'pymupdf']

# loop each parser
for parser_name in parser_name_list:

    # grab HTML
    p_html = p_jsonl_root / f'joint_to_{parser_name}/parsed_pdfs'
    jsonl_files = [p_html / f for f in os.listdir(p_html)]
    
    # each parser
    pdf_path_list = []
    pdf_text_list = []

    # status
    print(f'Parser: {parser_name} w/ {len(jsonl_files)} jsonl files.')

    # jsonl
    for jsonl_file in jsonl_files:
        # open
        with open(jsonl_file, 'r') as f:
            for line in f:
                data = json.loads(line)
                # extract path / text
                pdf_path = data['path']
                decoded_text = data['text']
                # append 
                pdf_path_list.append(pdf_path)
                pdf_text_list.append(decoded_text)

    # append to to dict
    all_dict[parser_name] = {'path' : pdf_path_list, 'text' : pdf_text_list}

# index : available PDFs (sorted by parent, filename)
all_pdf_paths = [item for sublist in [all_dict[k]['path'] for k in all_dict.keys()] for item in sublist]
index_set  = {pdf for pdf in all_pdf_paths}
index_list = [Path(idx) for idx in index_set if '.ipynb_checkpoints' not in idx]
sorted_index_list = sorted(index_list, key=lambda p: (Path(p).parent, Path(p).name))


# setup DataFrame
df = pd.DataFrame(index=sorted_index_list, columns=parser_name_list)

# Iterate over parser_name_list
for parser_name in parser_name_list:
    # Extract the corresponding dictionary from all_dict
    paths = all_dict[parser_name]['path']
    texts = all_dict[parser_name]['text']
    
    # Create a temporary DataFrame with paths as the index and texts as the data
    temp_df = pd.DataFrame(texts, index=paths, columns=[parser_name])
    
    # Update the main DataFrame df with the data from temp_df
    df.update(temp_df)

# Optionally, fill any remaining NaN values with a default value (e.g., empty string)
if fill_na_with_empty_str:
    df.fillna('', inplace=True)

# store
df.to_csv(store_path, sep='|')

Parser: html w/ 6 jsonl files.
Parser: marker w/ 440 jsonl files.
Parser: nougat w/ 388 jsonl files.
Parser: pymupdf w/ 4 jsonl files.


In [ ]:
df.to_csv('./database/decoded_text_across_parsers.csv', sep='|')

In [ ]:
# BLEU score
bleu_score = sentence_bleu([reference], candidate)
print(f'BLEU Score: {bleu_score:.2f}')

In [ ]:
# ROUGE score

reference = "This is a test."
candidate = "This is test."

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference, candidate)
print(scores)

In [ ]:
# CER

reference = "This unique is a test"
candidate = "This is test"

cer = jiwer.cer(reference, candidate)
print(f'CER: {cer:.2f}')